**Representation-based Language Models**
---

文章とか文字列を埋め込みなどを紹介してくれる章。

# ![](https://i.gyazo.com/55778936ee06eeba85ea972d8adc801e.png)

データに対して良い埋め込み、表現などが言語モデルの成功に大きく左右するという考え。

# ![](https://i.gyazo.com/0cc1f094057250abe3c0481887b732d4.png)

1はベクトルの成分に複素数が入っている場合の同値関係。

# ![](https://i.gyazo.com/3a2e0b826db7928c4b43f5de3bcf95c3.png)

ヒルベルト空間は距離と角度を考えられるような空間 (例: 二次元ユーグリッド空間)

上の定義を見ると、各要素の和が発散しないようなものをヒルベルト空間と呼ぶ？

任意の内積空間はヒルベルト空間に変換できるらしい。

# ![](https://i.gyazo.com/83f30b4ea13e667ac2384c59607bb015.png)

$\Sigma$の1要素につき、D次元のベクトルで表すよということ。

embeddingの仕方でone-hotの方法あるけど、sparseすぎるため、異なる文字間での類似度が常に0になって、意味的な類似度を捉えることができない。

なので、one-hotみたいに、|文字数|の数だけ行を持てばいいのではなくて、任意のトークン列を受け取って、ベクトルを返すようなものを考える、Attentionなど？

# ![](https://i.gyazo.com/16ca4eb5a1b51c47bb026aafc3558b91.png)

上のものが任意の文字列受け取って、ベクトル空間に移すもの。

$\operatorname{enc}(\overline{\boldsymbol{y}})$と、文字レベルの埋め込み$e(\bar{y})$ の内積はE enc $(\overline{\boldsymbol{y}})$のように表すようにして、これは型がEenc $(\bar{y})$なので、内積ベクトルをスコア(logits)とよぶ。これにsoftmaxなどをかければ、言語モデルのような取り扱いもできる？

---

**Projecting onto the Simplex**

上ではどうやって文章などがencodeされていてるかを議論したけど、ここからはそれらの埋め込みを使った言語モデルの作り方などを説明する。

**ふんわりとした理解になってしまうが、上でやってきたのは、文章をembeddingしたときに、どのように最終的に言語モデルに適用するかが不明だったが、そこに、文字のencodingを追加することによって、auto-regressive modelのような式になって解決する？**

どのように上のencodeされたものを言語モデルかにするのは、一番簡単な方法はカテゴリカル分布、次にsoftmax分布。

# ![](https://i.gyazo.com/11f547c814fa665e65ee2b439d7a5e0b.png)
# ![](https://i.gyazo.com/44e00b67af105d568ac65ba7f34dca4e.png)

**softmax関数は最適化問題に書き換えれるらしい、初知り。**
# ![](https://i.gyazo.com/ec53a44ab2d0c3a9a69679111d6626d8.png)